
# Asset Management on IBM Cloud Pak for Data

We are going to use this notebook to demonstrate how assets can be moved between deployment spaces using the Watson Machine Learning service. This approach may be useful in scenarios where there are separate environments used in the model development and delivery lifecycle. For example, you may have one space for development assets, one space to use as a staging/testing environment, and one space where your production assets are deployed. These spaces might all have different levels of access/authorization controls for different roles/users.

## 1.0 Install required packages


There are a couple of Python packages we will use in this notebook.

- We are using the <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> and will be interacting with it using the [ibm-watson-machine-learning](https://pypi.python.org/pypi/ibm-watson-machine-learning) library available in PyPI repository.

In [ ]:
!pip uninstall watson-machine-learning-client -y | tail -n 1
!pip uninstall watson-machine-learning-client-V4 -y | tail -n 1

!pip install --upgrade ibm-watson-machine-learning==1.0.79 --user --no-cache | tail -n 1

In [ ]:
import json
from ibm_watson_machine_learning import APIClient

## 2.0 Instantiate Watson Machine Learning Client

To authenticate the Watson Machine Learning service on IBM Cloud, you need to provide a platform `api_key` and an endpoint URL. Where the endpoint URL is based on the `location` of the WML instance. To get these values you can use either the IBM Cloud CLI or the IBM Cloud UI.


#### IBM Cloud CLI

You can use the [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) to create a platform API Key and retrieve your instance location.

- To generate the Cloud API Key, run the following commands:
```
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME
```
  - Copy the value of `api_key` from the output.


- To retrieve the location of your WML instance, run the following commands:
```
ibmcloud login --apikey API_KEY -a https://cloud.ibm.com
ibmcloud resource service-instance "WML_INSTANCE_NAME"
```
> Note: WML_INSTANCE_NAME is the name of your Watson Machine Learning instance and should be quoted in the command.

  - Copy the value of `Location` from the output.

#### IBM Cloud UI

To generate Cloud API key:
- Go to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). 
- From that page, click your name in the top right corner, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. 
- Give your key a name and click **Create**, then copy the created key and to use it below.

To retrieve the location of your WML instance:
- Go to the [**Resources List** section of the Cloud console](https://cloud.ibm.com/resources).
- From that page, expand the **Services** section and find your Watson Machine Learning Instance.
- Based on the Location displayed in that page, select one of the following values for location variable:
|Displayed Location|Location|
|-|-|
|Dallas|us-south|
|London|eu-gb|
|Frankfurt|eu-de|
|Tokyo|jp-tok|


<font color='red'><<**ENTER YOUR IBM CLOUD `api_key` AND MACHINE LEARNING SERVICE INSTANCE `location` IN THE CELL BELOW**>></font>

In [ ]:
API_KEY = 'PASTE YOUR PLATFORM API KEY HERE'
LOCATION = 'PASTE YOUR INSTANCE LOCATION HERE'

In [ ]:
wml_credentials = {
    "apikey": API_KEY,
    "url": 'https://' + LOCATION + '.ml.cloud.ibm.com'
}
wml_client = APIClient(wml_credentials)

## 3.0 Get Assets from `Development` Deployment Spaces

We will find the `GUID` for the deployment space you want to use and set it as the default space for the client.

In [ ]:
wml_client.spaces.list()

<font color='red'><<**UPDATE THE VARIABLE `EXPORT_SPACE_ID`  TO THE GUID OF THE DEPLOYMENT SPACE YOU ARE PULLING MODELS FROM.**>></font>

> Note: You should copy the ID of your deployment space from the output of the previous cell to the variable in the next cell. 

In [ ]:
EXPORT_SPACE_ID = 'PASTE YOUR DEPLOYMENT SPACE GUID HERE'

In [ ]:
# Now set the default space to the GUID for your deployment space. If this is successful, you will see a 'SUCCESS' message.
wml_client.set.default_space(EXPORT_SPACE_ID)

### 3.1 Get all models in the space

Start by listing all the models in your deployment space. Though we could export more than models, for this exercise we only want to move ML model assets.

In [ ]:
models_details = wml_client.repository.list_models()

<font color='red'><<**UPDATE THE VARIABLE `MODEL_ID_TO_EXPORT`  TO THE ID OF THE MODEL YOU ARE GOING TO EXPORT.**>></font>

> Note: You should copy the ID of your MODEL from the output of the previous cell to the variable in the next cell. 

In [ ]:
MODEL_ID_TO_EXPORT = 'PASTE YOUR MODEL ID HERE'

### 3.2 Use Export Function

We will use the export method in the Watson Machine Learning client. The export method allows users to create a job that will extract assets from a deployment space. 

In [ ]:
help(wml_client.export_assets.start)

#### 3.2.1 Start the export process

The export process can be used to export either: specific assets (using a collection of ASSET_IDS), specific asset types (i.e all models, all data files, etc) or all assets in the space.

We are going to export a single model (the `MODEL_ID_TO_EXPORT` we specified above), but if you wanted to export more than one model ids, you need to provide them as array like client.export_assets.ConfigurationMetaNames.ASSET_IDS: [model_id1, model_id2]

In [ ]:
metadata = { wml_client.export_assets.ConfigurationMetaNames.NAME: "export_model",
             wml_client.export_assets.ConfigurationMetaNames.ASSET_IDS: [MODEL_ID_TO_EXPORT]                                                                         
           }

details = wml_client.export_assets.start(meta_props=metadata, space_id=EXPORT_SPACE_ID)
print(json.dumps(details, indent=2))

export_job_id = details[u'metadata'][u'id']
print("Export Job ID: ", export_job_id)

#### 3.2.2 Monitor the export process

In [ ]:
import time

start_time = time.time()
diff_time = start_time - start_time
while True and diff_time < 10 * 60:
    time.sleep(3)
    response = wml_client.export_assets.get_details(export_job_id, space_id=EXPORT_SPACE_ID)
    state = response[u'entity'][u'status'][u'state']
    print(state)
    if state == 'completed' or state == 'error' or state == 'failed':
        break
    diff_time = time.time() - start_time

print(json.dumps(response, indent=2))

#### 3.2.3 Get Exported Content

You can now get the file with the exported assets.

In [ ]:
import os
import uuid

export_dir = 'EXPORT_DATA'

if not os.path.isdir(export_dir):
    os.mkdir(export_dir)
    
export_file_name = 'exported_content_' + str(uuid.uuid4()) + '.zip'
    
export_file_path = os.path.join(export_dir, export_file_name)

details = wml_client.export_assets.get_exported_content(export_job_id, 
                                                    space_id = EXPORT_SPACE_ID, 
                                                    file_path = export_file_path)

print(details)

In [ ]:
!ls -al EXPORT_DATA

## 4.0 Push Assets to target `Staging` Deployment Space

We will find the `GUID` for the deployment space you want to target as your "staging" or "testing" space where the assets will be imported.

In [ ]:
wml_client.spaces.list()

<font color='red'><<**UPDATE THE VARIABLE `IMPORT_SPACE_ID`  TO THE GUID OF THE DEPLOYMENT SPACE YOU ARE PUSHING MODELS TO.**>></font>

> Note: You should copy the ID of your deployment space from the output of the previous cell to the variable in the next cell. 

In [ ]:
IMPORT_SPACE_ID = 'PASTE YOUR DEPLOYMENT SPACE GUID HERE'

### 4.1 Use the import Function

Now that we have the assets exported, we can export them to another deployment space. In a real world scenario, we may be pushing the exported assets to a source control system, maybe even use some CI/CD tooling to then move them to the target deployment environment. For this simple example, we are just going to move the models to another deployment space.

We will be using the Watson Machine Learning client `import_assets` API. This service will kick of an asynchronous job that can be monitored and managed.


#### 4.1.1 Start the import process

In [ ]:
details = wml_client.import_assets.start(file_path=export_file_path,
                                     space_id=IMPORT_SPACE_ID)
print(json.dumps(details, indent=2))

import_job_id = details[u'metadata'][u'id']
print("Import job id: ", import_job_id)

#### 4.2 Monitor the import process

We will monitor the import job until it either completes, fails or errors out.

In [ ]:
import time

start_time = time.time()
diff_time = start_time - start_time
while True and diff_time < 10 * 60:
    time.sleep(3)
    response = wml_client.import_assets.get_details(import_job_id,
                                                space_id=IMPORT_SPACE_ID)
    state = response[u'entity'][u'status'][u'state']
    print(state)
    if state == 'completed' or state == 'error' or state == 'failed':
         break
    diff_time = time.time() - start_time

print(json.dumps(response, indent=2))

#### 4.3 List Assets in Target Space

We can verify the import process by listing the assets in the target deployment space.

In [ ]:
# Now set the default space to the GUID for your deployment space. If this is successful, you will see a 'SUCCESS' message.
wml_client.set.default_space(IMPORT_SPACE_ID)

In [ ]:
print("{}List of models: {}".format('\n', '\n'))
wml_client.repository.list_models()
details = wml_client.repository.get_model_details()

## [OPTIONAL] 5.0 Deploy and score the imported model

As an optional exercise, we could create an online deployment and test the imported model. The cells below are set to a raw cell type since this is an optional section. If you wish to run this exercise, change the type to code by selecting each cell and selecting the `Code` type from the "Format" drop down meny in the toolbar.

## 6.0 Clean up 

You could now clean up your deployment spaces. The code below will delete the deployment spaces. This is left commented out in case you want to keep your deployment space. If you are done with all the labs and want to delete the spaces, uncomment the code (remove the `#` character and run the cell.

In [ ]:
#wml_client.export_assets.delete(export_job_id, space_id=EXPORT_SPACE_ID)
#wml_client.import_assets.delete(import_job_id, space_id=IMPORT_SPACE_ID)
#wml_client.spaces.delete(EXPORT_SPACE_ID)
#wml_client.spaces.delete(IMPORT_SPACE_ID)

## Credits / Acknowledgements

Adapted from - https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20export%20and%20import.ipynb

- Original Authors: Mithun - *vbmithun@in.ibm.com**, Software Engineer

Copyright © 2020, 2021 IBM. This notebook and its source code are released under the terms of the MIT License.